In [10]:
import numpy as np
import os
import matplotlib.pyplot as plt
import h5py
from scipy import interpolate
from ccfj import GetStationPairs
import pandas as pd
import yaml
import geopandas as gp
import openpyxl
from openpyxl import load_workbook
from concurrent.futures import ThreadPoolExecutor
from scipy.interpolate import make_interp_spline

In [11]:
flag_project = 1 # 0--regular ; 1--repartrition

In [12]:
if flag_project == 0:
    file_project = 'a-project.yml'
    with open('a-project.yml', 'r', encoding='utf-8') as f:
        proj = yaml.load(f.read(), Loader=yaml.FullLoader)
    name_project = proj['name']
    #name_project = 'project/output_FJSJ_16-01/'               # Harmon server
elif flag_project == 1:
    file_project = 'a-project_repar.yml'
    with open('a-project_repar.yml', 'r', encoding='utf-8') as f:
        proj = yaml.load(f.read(), Loader=yaml.FullLoader)
    name_project = proj['name']
    #name_project = 'project_repartrition/repartrition_01-03/'               # Harmon server


In [13]:
with open('0_config.yml', 'r', encoding='utf-8') as f:
    dir_config = yaml.load(f.read(), Loader=yaml.FullLoader)
dir_project_workspace = dir_config['dir_project_workspace']
dir_CC_workspace = dir_config['dir_CC_workspace']
print('dir_CC_workspace: ', dir_CC_workspace)
print('dir_project_workspace: ', dir_project_workspace)
dir_project = os.path.join(dir_project_workspace, name_project)
print('dir_project: ', dir_project)

dir_CC_workspace:  /shdisk/rem2/Harmon/F-J/San/
dir_project_workspace:  /shdisk/rem2/Harmon/F-J/San/
dir_project:  /shdisk/rem2/Harmon/F-J/San/project_repartition_v3.0/output_repar_01-01/


In [14]:
filename = dir_project+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = dir_project+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary

In [15]:
dir_disp_autopick = dir_project + info_basic['dir_disp']

In [16]:
key_subworks = info_basic['key_subworks']
key_subworks

['07-03--1',
 '09-03--2',
 '10-03--3',
 '11-03--4',
 '12-03--5',
 '13-03--6',
 '14-03--7',
 '16-03--8',
 '17-03--9',
 '18-03--10',
 '19-03--11',
 '20-03--12',
 '21-03--13',
 '22-03--14',
 '23-03--15',
 '24-03--16',
 '25-03--17',
 '26-03--18',
 '27-03--19',
 '28-03--20',
 '29-03--21',
 '31-03--22',
 '32-03--23',
 '34-03--24',
 '36-03--25',
 '37-03--26',
 '38-03--27',
 '39-03--28',
 '40-03--29',
 '41-03--30',
 '43-03--31',
 '45-03--32',
 '46-03--33',
 '47-03--34',
 '48-03--35',
 '49-03--36',
 '52-03--37',
 '53-03--38',
 '54-03--39',
 '07-04--40',
 '08-04--41',
 '09-04--42',
 '10-04--43',
 '11-04--44',
 '12-04--45',
 '14-04--46',
 '15-04--47',
 '16-04--48',
 '18-04--49',
 '19-04--50',
 '20-04--51',
 '21-04--52',
 '22-04--53',
 '23-04--54',
 '24-04--55',
 '25-04--56',
 '26-04--57',
 '27-04--58',
 '28-04--59',
 '29-04--60',
 '30-04--61',
 '31-04--62',
 '32-04--63',
 '33-04--64',
 '36-04--65',
 '39-04--66',
 '40-04--67',
 '43-04--68',
 '45-04--69',
 '46-04--70',
 '47-04--71',
 '48-04--72',
 

### create workspace for Dispernet

In [17]:
info_basic['dir_inv_dispernet'] = 'inv_dispernet/'
dir_inv = dir_project + info_basic['dir_inv_dispernet']
if not os.path.exists(dir_inv):
    os.makedirs(dir_inv)
with open(dir_project+'Basic_info.yml', 'w', encoding='utf-8') as f:
   yaml.dump(data=info_basic, stream=f, allow_unicode=True)

In [18]:
dir_disp = dir_inv + 'data/'
if not os.path.exists(dir_disp):
    os.makedirs(dir_disp)

### Autopick

In [19]:
flag_f = [4,15]

In [20]:
def transfer_autopick(key_subwork):
    global dir_disp_autopick, dir_disp, flag_f
    print(key_subwork)
    ff_pick = np.loadtxt(dir_disp_autopick+'autopick_'+key_subwork+'.txt',delimiter=',')[:,0]
    cc_pick = np.loadtxt(dir_disp_autopick+'autopick_'+key_subwork+'.txt',delimiter=',')[:,1]
    f_start = flag_f[0]
    f_end = flag_f[1]
    ff_this = ff_pick[(ff_pick>=f_start) & (ff_pick<=f_end)]
    cc_this = cc_pick[(ff_pick>=f_start) & (ff_pick<=f_end)]
    np.savetxt(dir_disp+'autopick_'+key_subwork+'.txt',np.c_[ff_this,cc_this],fmt='%f,%f')

In [21]:
for key_subwork in key_subworks:
    print(key_subwork)
    transfer_autopick(key_subwork)

07-03--1
07-03--1
09-03--2
09-03--2
10-03--3
10-03--3
11-03--4
11-03--4
12-03--5
12-03--5
13-03--6
13-03--6
14-03--7
14-03--7
16-03--8
16-03--8
17-03--9
17-03--9
18-03--10
18-03--10
19-03--11
19-03--11
20-03--12
20-03--12
21-03--13
21-03--13
22-03--14
22-03--14
23-03--15
23-03--15
24-03--16
24-03--16
25-03--17
25-03--17
26-03--18
26-03--18
27-03--19
27-03--19
28-03--20
28-03--20
29-03--21
29-03--21
31-03--22
31-03--22
32-03--23
32-03--23
34-03--24
34-03--24
36-03--25
36-03--25
37-03--26
37-03--26
38-03--27
38-03--27
39-03--28
39-03--28
40-03--29
40-03--29
41-03--30
41-03--30
43-03--31
43-03--31
45-03--32
45-03--32
46-03--33
46-03--33
47-03--34
47-03--34
48-03--35
48-03--35
49-03--36
49-03--36
52-03--37
52-03--37
53-03--38
53-03--38
54-03--39
54-03--39
07-04--40
07-04--40
08-04--41
08-04--41
09-04--42
09-04--42
10-04--43
10-04--43
11-04--44
11-04--44


12-04--45
12-04--45
14-04--46
14-04--46
15-04--47
15-04--47
16-04--48
16-04--48
18-04--49
18-04--49
19-04--50
19-04--50
20-04--51
20-04--51
21-04--52
21-04--52
22-04--53
22-04--53
23-04--54
23-04--54
24-04--55
24-04--55
25-04--56
25-04--56
26-04--57
26-04--57
27-04--58
27-04--58
28-04--59
28-04--59
29-04--60
29-04--60
30-04--61
30-04--61
31-04--62
31-04--62
32-04--63
32-04--63
33-04--64
33-04--64
36-04--65
36-04--65
39-04--66
39-04--66
40-04--67
40-04--67
43-04--68
43-04--68
45-04--69
45-04--69
46-04--70
46-04--70
47-04--71
47-04--71
48-04--72
48-04--72
49-04--73
49-04--73
52-04--74
52-04--74
53-04--75
53-04--75
54-04--76
54-04--76
07-05--77
07-05--77
09-05--78
09-05--78
11-05--79
11-05--79
12-05--80
12-05--80
13-05--81
13-05--81
14-05--82
14-05--82
15-05--83
15-05--83
16-05--84
16-05--84
18-05--85
18-05--85
19-05--86
19-05--86
20-05--87
20-05--87
21-05--88
21-05--88
22-05--89
22-05--89
23-05--90
23-05--90
24-05--91
24-05--91
25-05--92
25-05--92
26-05--93
26-05--93
27-05--94
27-05--94
